In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import sympy as sp
import numpy as np
from Modules.classes import Operator, Coefficient, Term, Expression
from Modules.solver import *

In [18]:
hbar = Coefficient('hbar', order=0)
omega = Coefficient('omega', order=0)
Omega_z = Coefficient('Omega_z', order=0)
g = Coefficient('g', order=1)

a = Operator('a', subspace='oscillator', is_infinite=True, mat_representation=sp.Matrix([[0, 1, 0], [0, 0, 1], [0, 0, 0]]))
a_dag = Operator('{a^\\dagger}', subspace='oscillator', is_infinite=True, mat_representation=sp.Matrix([[0, 0, 0], [1, 0, 0], [0, 1, 0]]))

a.add_commutation_relation(a_dag, 1)
a_dag.add_commutation_relation(a, -1)

X = Operator('sigma_x', mat_representation=sp.Matrix([[0, 1], [1, 0]]), subspace='spin')
Z = Operator('sigma_z', mat_representation=sp.Matrix([[1, 0], [0, -1]]), subspace='spin')
Y = Operator('tau_y', mat_representation=sp.Matrix([[0, -1j], [1j, 0]]), subspace='charge')

H = hbar * omega * a_dag * a + hbar * Omega_z * 0.5 * Z + hbar * g * (a + a_dag) * X
H


hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar a sigma_x + g*hbar {a^\dagger} sigma_x

In [19]:
H_new, subspaces = H.domain_expansion()

In [20]:
Vk = H_new.group_by_diagonal()[False].group_by_order()[1]
Vk
get_ansatz(Vk)[0]

\varepsilon_{(1)} S^{(0)}_1 + \varepsilon_{(1)} S^{(1)}_1 a + \varepsilon_{(1)} S^{(2)}_1 {a^\dagger}

In [21]:
subspaces

['spin']

In [22]:
H_new

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar sigma_x a + g*hbar sigma_x {a^\dagger}

In [23]:
H_new.terms[0].info

{'coeff': [hbar, omega, 1, 1, 1, 1, 1, 1], 'oscillator': [{a^\dagger}, a]}

In [24]:
S, symbols_for_s = get_ansatz(H_new)
S.terms[0].info

{'coeff': [1, \varepsilon_{(1)}], 'finite': [S^{(0)}_1]}

In [25]:
for key, value in ((H_new.group_by_order()[0] | S ) + H_new.group_by_order()[1].group_by_diagonal()[False]).group_by_infinite_terms().items():
    print(f'\n\n\n{key}:\n')
    print([sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) for term in value.terms])
    mats = sp.Add(*[sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) * term.mat_representation["finite"] for term in value.terms])
    display(mats)
    display(sp.solve(mats, symbols_for_s))




1:

[-Omega_z*\varepsilon_{(1)}*hbar/2, Omega_z*\varepsilon_{(1)}*hbar/2]


Matrix([
[                                          0, Omega_z*S^{(0)}_1_1*\varepsilon_{(1)}*hbar],
[-Omega_z*S^{(0)}_1_2*\varepsilon_{(1)}*hbar,                                          0]])

{S^{(0)}_1_1: 0, S^{(0)}_1_2: 0}




1 a:

[-\varepsilon_{(1)}*hbar*omega, -Omega_z*\varepsilon_{(1)}*hbar/2, Omega_z*\varepsilon_{(1)}*hbar/2, g*hbar]


In [ ]:
from IPython.display import display, Math
def solver(H, order):
    H_expanded = H.domain_expansion()[0]
    H_0 = H_expanded.group_by_order()[0]
    H_final = deepcopy(H_0)
    Bk = 0

    S = 0

    for k in range(1, order+1):
        print(f"Order {k}")
        H_below_k = sum(H_expanded.group_by_order().get(j, 0) for j in range(k+1))
        Vk = H_below_k.group_by_diagonal()[False].group_by_order().get(k, 0)
        S_k, symbols_sk = get_ansatz(Vk + Bk)
        solution_sk = dict(zip(symbols_sk, [0 for _ in symbols_sk]))
        #display(Bk)
        for key, value in ((H_0 | S_k) + (Vk + Bk)).group_by_infinite_terms().items():
            print("Im here 1")
            mats = sp.Add(*[sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) * term.mat_representation["finite"] for term in value.terms])
            solution_sk.update(sp.solve(mats, symbols_sk))
        
        S_k_solved = 0
        for term in S_k.terms:
            print("Im here 2")
            term.mat_representation["finite"] = term.mat_representation["finite"].subs(solution_sk)
            if sum(sp.Abs(term.mat_representation["finite"])) == 0:
                continue
            S_k_solved += term
        S += S_k_solved
        display(S)
        tmp_H = float(1 / sp.factorial(k)) * H_below_k.nested_commutator(S, k)
        H_final += tmp_H.group_by_diagonal().get(True, Expression(Term(Coefficient(0)))).group_by_order().get(k, 0)
        display(H_final)
        Bk = tmp_H.group_by_diagonal()[False].group_by_order()[k+1]
    
    return H_final


In [ ]:
H_expanded = H.domain_expansion()[0]
H_expanded.group_by_order()[0]

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z

In [ ]:
H

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar a sigma_x + g*hbar {a^\dagger} sigma_x

In [ ]:
solver(H, 2)

Order 1
Im here 1
Im here 1
Im here 1
Im here 2
Im here 2
Im here 2


\varepsilon_{(1)} S^{(1)}_1 a + \varepsilon_{(1)} S^{(2)}_1 {a^\dagger}

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z

Order 2
Im here 1
Im here 1
Im here 1
Im here 1
Im here 1
Im here 2
Im here 2
Im here 2
Im here 2
Im here 2


\varepsilon_{(1)} S^{(1)}_1 a + \varepsilon_{(1)} S^{(2)}_1 {a^\dagger} + \varepsilon_{(1)} S^{(0)}_2 + \varepsilon_{(2)} S^{(1)}_2 a**2 + \varepsilon_{(2)} S^{(2)}_2 a*{a^\dagger} + \varepsilon_{(2)} S^{(3)}_2 {a^\dagger}*a + \varepsilon_{(2)} S^{(4)}_2 {a^\dagger}**2

KeyboardInterrupt: 

In [ ]:
for i in sp.Matrix([[1,2],[3,4]]):
    print(i)

# Trying out sympy module

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from sympy.physics.quantum import Dagger

In [3]:
sx = RDOperator("sigma_x", subspace = "spin", dim = 2, matrix=sp.Matrix([[0,1],[1,0]]))
sz = RDOperator("sigma_z", subspace = "spin", dim = 2, matrix=sp.Matrix([[1,0],[0,-1]]))


In [4]:
names = ["sigma_x", "sigma_z", "tau_x"]
subspaces = ["spin", "spin", "charge"]
dims = [2, 2, 2]
matrices = [sp.Matrix([[0,1],[1,0]]), sp.Matrix([[1,0],[0,-1]]), sp.Matrix([[0,1],[1,0]])]

sx, sz, tx = get_finite_operators(names, subspaces, dims, matrices)
a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("Omega_z", order = 0)
g = RDsymbol("g", order = 1)
f = RDsymbol("f", order = 2)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz + hbar * g * (a + ad) * sx + hbar * f * tx
H

hbar*Omega_z*sigma_z/2 + hbar*f*tau_x + hbar*g*(a + {a^\dagger})*sigma_x + hbar*omega*{a^\dagger}*a

In [5]:
H_ordered = group_by_order(H)
H0 = H_ordered.get(0, 0)
Vk = H_ordered.get(1, 0)

In [6]:
def make_matrix(name='a', n=2):
    s = sp.Matrix([[sp.symbols(f'{name}_'+ '{' +f'{i}{j}' + '}') for j in range(n)] for i in range(n)])
    return s

In [7]:
S1 = RDOperator('S1', subspace='finite', dim=2, matrix=make_matrix('S1', 4))
S2 = RDOperator('S2', subspace='finite', dim=2, matrix=make_matrix('S2', 4))
S1_total = S1 * a + S2 * ad
equations_to_solve = group_by_infinite_operators(expand_commutator(Commutator(H0, S1_total)).doit() + Vk)
equations = []
for _, value in equations_to_solve.items():
    equations.append(value)
matrices = [get_matrix(eq) for eq in equations if eq != 0]

-1


ValueError: Invalid reshape parameters 2 2

In [ ]:
sz.dim

4

In [13]:
Commutator(ad, omega).doit()

0